In [1]:
from pathlib import Path
import pandas as pd

# --- Config ---
PERSON = "p00"  # change to your subject
REPO_ROOT = Path.cwd()  # or manually point to your repo root
DATASET_ROOT = REPO_ROOT / "dataset" / "MPIIFaceGaze"
OUT_DIR = REPO_ROOT / "outputs" / "preprocessed" / PERSON

META_PATH = OUT_DIR / "meta.csv"
ANNOT_PATH = DATASET_ROOT / PERSON / f"{PERSON}.txt"
META_EXPANDED_PATH = OUT_DIR / "meta_expanded.csv"

# --- Load existing meta ---
meta = pd.read_csv(META_PATH)
print("Current meta columns:", meta.columns.tolist())


Current meta columns: ['out_path', 'rel_path', 'file_name', 'eye_label', 'x0', 'y0', 'x1', 'y1', 'out_size', 'pad_scale']


In [2]:
def parse_annotation_line(line: str):
    t = line.strip().split()
    if len(t) < 28: return None
    rel_path = t[0]
    sx, sy = float(t[1]), float(t[2])
    rvec = tuple(map(float, t[15:18]))
    tvec = tuple(map(float, t[18:21]))
    fc   = tuple(map(float, t[21:24]))
    gt   = tuple(map(float, t[24:27]))
    eye_label = t[27].lower()

    # Gaze vectors
    gaze_dx, gaze_dy, gaze_dz = gt[0]-fc[0], gt[1]-fc[1], gt[2]-fc[2]
    n = (gaze_dx**2 + gaze_dy**2 + gaze_dz**2)**0.5
    gaze_ux, gaze_uy, gaze_uz = (gaze_dx/n, gaze_dy/n, gaze_dz/n) if n>0 else (0,0,0)

    return {
        "rel_path": rel_path,
        "eye_label": eye_label,
        "screen_x": sx, "screen_y": sy,
        "rvec_x": rvec[0], "rvec_y": rvec[1], "rvec_z": rvec[2],
        "tvec_x": tvec[0], "tvec_y": tvec[1], "tvec_z": tvec[2],
        "fc_x": fc[0], "fc_y": fc[1], "fc_z": fc[2],
        "gt_x": gt[0], "gt_y": gt[1], "gt_z": gt[2],
        "gaze_dx": gaze_dx, "gaze_dy": gaze_dy, "gaze_dz": gaze_dz,
        "gaze_ux": gaze_ux, "gaze_uy": gaze_uy, "gaze_uz": gaze_uz
    }

# Build annotation DataFrame
rows = []
with open(ANNOT_PATH, "r", encoding="utf-8") as f:
    for ln in f:
        rec = parse_annotation_line(ln)
        if rec: rows.append(rec)
ann_df = pd.DataFrame(rows)
print("Annotation rows parsed:", len(ann_df))


Annotation rows parsed: 2927


In [3]:
# Ensure eye_label is present in meta
if "eye_label" not in meta.columns:
    def infer_eye(fn: str):
        s = str(fn).lower()
        if s.endswith("_l.png"): return "left"
        if s.endswith("_r.png"): return "right"
        return None
    meta["eye_label"] = meta["file_name"].map(infer_eye)

# Merge on rel_path + eye_label
merged = pd.merge(meta, ann_df, on=["rel_path","eye_label"], how="left", validate="m:1")

# Reorder columns
front_cols = ["out_path","rel_path","file_name","eye_label",
              "x0","y0","x1","y1","out_size","pad_scale"]
extra_cols = ["screen_x","screen_y",
              "rvec_x","rvec_y","rvec_z",
              "tvec_x","tvec_y","tvec_z",
              "fc_x","fc_y","fc_z",
              "gt_x","gt_y","gt_z",
              "gaze_dx","gaze_dy","gaze_dz",
              "gaze_ux","gaze_uy","gaze_uz"]

merged = merged[front_cols + extra_cols]

# Save
merged.to_csv(META_EXPANDED_PATH, index=False, float_format="%.6f")
print("Wrote expanded meta:", META_EXPANDED_PATH)
print(merged.head())


Wrote expanded meta: c:\Users\jayfo\OneDrive\Desktop\EyeTrackpad\outputs\preprocessed\p00\meta_expanded.csv
                                            out_path        rel_path  \
0  C:\Users\jayfo\OneDrive\Desktop\EyeTrackpad\ou...  day01/0005.jpg   
1  C:\Users\jayfo\OneDrive\Desktop\EyeTrackpad\ou...  day01/0030.jpg   
2  C:\Users\jayfo\OneDrive\Desktop\EyeTrackpad\ou...  day01/0031.jpg   
3  C:\Users\jayfo\OneDrive\Desktop\EyeTrackpad\ou...  day01/0038.jpg   
4  C:\Users\jayfo\OneDrive\Desktop\EyeTrackpad\ou...  day01/0063.jpg   

              file_name eye_label   x0   y0   x1   y1  out_size  pad_scale  \
0  day01_0005.jpg_R.png     right  577  327  654  404        64        0.4   
1  day01_0030.jpg_L.png      left  621  340  711  431        64        0.4   
2  day01_0031.jpg_L.png      left  722  347  802  427        64        0.4   
3  day01_0038.jpg_R.png     right  575  369  661  456        64        0.4   
4  day01_0063.jpg_R.png     right  531  332  603  404        64      